# Парсер вакансий hh, telegram links

Планируется след. модель исползования - выгружаю вакансии за определенный период предобрабатываю, сохраняю в отдельный файл для какой нибудь аналитики.   
Т.е вся логика ноутбука сводится к одному такому циклу работы.

In [1]:
#!pip install pyrogram tgcrypto
#!pip install python-dotenv
#C:\Users\Evgeny\Jupyter_notebooks\Yandex\web_homework

## Сбор данных

In [2]:
import os
import re
import pandas as pd
import numpy as np
import json
import pyrogram
import requests


#from bs4 import BeautifulSoup
from dotenv import load_dotenv

from pyrogram import Client
from datetime import datetime


### Telegram

Баш на windows не работает корректно, сыплет вопросами и ошибками, хотя установлен и включен. Запускаю скрипт питоном машины, тк. пакеты мы все равно устанавливаем не в ядро ноутбука, а в питон машины. Так же пути с точкой, вида - '.env', не работают. Думаю все косяки связанны с windows (╯°□°）╯︵ ┻━┻. По этому я немного поправил скрипты и на моей машине это работает.  
┬─┬ ノ( ゜-゜ノ)

#### Выгрузка сообщений

'telegram.csv' используется как временный рабочий фрейм с загруженными сообщениями из телеграма, в котором информация обрабатывается затем попадает в основной DF и 'telegram.csv' удаляется.

In [3]:
#Здесь задаю каналы и кол-во сообщений
target_chanels = ['datajob', 'foranalysts', 'analyst_job', 'data_hr', 'analysts_hunter','biheadhunter', 'datasciencejobs']
arg1 = ','.join(target_chanels)
arg2 = 50

#Данные приложения из переменных среды
dotenv_path = os.path.join('env')

if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    
arg3 = os.environ['API_ID']
arg4 = os.environ['API_HASH']

command_line = f'-id {arg3} -hash {arg4} -t {arg1} -n {arg2}'

In [4]:
!python pyro_run.py $command_line

Pyrogram v1.2.9, Copyright (C) 2017-2021 Dan <https://github.com/delivrance>
Licensed under the terms of the GNU Lesser General Public License v3 or later (LGPLv3+)

Success:  C:\Users\Evgeny\Jupyter_notebooks\Yandex\web_homework\telegram.csv


In [5]:
telegram_raw = pd.read_csv('telegram.csv')

In [6]:
telegram_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   chat        250 non-null    object
 1   message_id  350 non-null    int64 
 2   date        350 non-null    int64 
 3   text        321 non-null    object
 4   entities    279 non-null    object
dtypes: int64(2), object(3)
memory usage: 13.8+ KB


Сообщения без структуры и текста

In [7]:
#telegram_raw[telegram_raw['entities'].isna()]
#telegram_raw[telegram_raw['chat'].isna()]

In [8]:
#Дропаю все сообщения без структуры, тк пока не разобрался что это за сообщения такие.
telegram_raw = telegram_raw.dropna(subset=['chat'])
telegram_raw = telegram_raw.dropna(subset=['entities'])

Сообщения можно различать по chat id + message_id. Так же из всей информации о чате пока что оставляю только id

In [9]:
#id канала
telegram_raw['id'] = telegram_raw['chat'].apply(lambda x: re.findall(r'-\d*,', x)[0][1:-1]) + telegram_raw['message_id'].apply(str)
#Название канала
telegram_raw['channel'] = telegram_raw['chat'].apply(lambda x: re.findall(r'\"username\": \"\w+\"', x)[0][13:-1])

In [10]:
telegram_raw = telegram_raw.drop('chat', axis=1)
telegram_raw = telegram_raw.drop('message_id', axis=1)

id приятней видеть первым

In [11]:
c_names = list(telegram_raw.columns)
c_names[0], c_names[-1]  = c_names[-1], c_names[0]
telegram_raw = telegram_raw[c_names]

#### Парсинг

Разметка сообщений телеграма оказалась крайней не надежной, по этому использую смесь из re и того что размечено в json для парсинга.

In [12]:
#Парсим контакты из текста сообщения, по разметке в джсон
#Пока что сделал так, но выглядит перегруженно и неочень
def parse_contact_links_debug(x):
    url_reg = 'http\S+'
    telegram_reg = '\S+t.me\S+'
    
    urls = list()
    emails = list()
    mentions = list()
    tags = list()
#    print(x['entities'])
    try:
        for i in json.loads(x['entities']):
#            print(i)
            offset = i['offset']
            lenght = i['length']
            #это могут быть ссылки и на телеграм и на ютуб и на сайт с вакансией
            if i['type'] == 'text_link':
                place = x['text'][offset : offset + lenght]
#                print(place)
                tmp = re.findall(url_reg, place)
                for link in tmp:
                    urls.append(link)
                tmp = re.findall(telegram_reg, place)
                for link in tmp:
                    mentions.append(link)
            #пока что верим что тут только эмейлы
            elif i['type'] == 'email':
                emails.append(x['text'][offset : offset + lenght])
            #ссылка на телеграм
            elif i['type'] == 'mention':
                mentions.append('https://t.me/' + x['text'][offset +1 : offset + lenght])#пропускаю @
            #это могут быть ссылки и на телеграм и на ютуб и на сайт с вакансией
            elif i['type'] == 'url':
                place = x['text'][offset -4 : offset + lenght]# увидел что у offset url встречается ошибка в 4 знака в смещеннии
#                print(place)
                tmp = re.findall(url_reg, place)
                for link in tmp:
#                    print(link)
                    urls.append(link)
                tmp = re.findall(telegram_reg, place)
                for link in tmp:
                    mentions.append(link)
            #теги
            elif i['type'] == 'hashtag':
                tmp = x['text'][offset : offset + lenght]
                tags.append(x['text'][offset : offset + lenght])
                
    except Exception as e:
        print(e)
        return None, None, None
    
    urls = (lambda x: x if(len(x) != 0) else None)(urls)
    emails = (lambda x: x if(len(x) != 0) else None)(emails)
    mentions = (lambda x: x if(len(x) != 0) else None)(mentions)
    tags = (lambda x: x if(len(x) != 0) else None)(tags)
    
    return urls, emails, mentions, tags

Получаю столбцы контактов и можно удалять entities столбец

In [13]:
tmp_df = telegram_raw.apply(parse_contact_links_debug, axis=1, result_type='expand')
tmp_df.columns = ['urls', 'emails', 'mentions', 'tags']

#telegram_raw = telegram_raw.drop('entities', axis=1)
telegram_raw = pd.concat([telegram_raw, tmp_df], axis=1)
telegram_raw = telegram_raw.drop('entities', axis=1)

На данном этапе сохраняю нужные данные в csv, что бы скриптом брать оттуда ссылки и парсить их. Потом обратно заджойнить по id.   
Планирую такого вида файл скармливать павукам и получать его же но с html. Думаю есть решения лучше чем csv например через json.  

In [14]:
f1 = telegram_raw['urls'].notna()
#f2 = telegram_raw[f1]['urls'].apply(lambda x: len(x) == 1)
#Нужные столбцы
columns = ['id', 'urls']

In [15]:
tmp_df = telegram_raw.loc[f1, columns]
#Проблема 1 - после сохранения лист становится строкой. . .
#Так же воможно испортились столбцы 'chat' и 'entities'

#Пока не придумал что делать с остальными ссылками
#по этому парсить буду 1ю ссылку в списке
tmp_df['urls'] = tmp_df['urls'].apply(lambda x: x[0])
tmp_df['desc_html'] = pd.NA
tmp_df.to_csv('tmp_urls.csv', index=False)

HTML вакансий будет собираться, Scrapy павуками. /╲/\╭( ͡° ͡° ͜ʖ ͡° ͡°)╮/\╱\

In [16]:
#!pip install Scrapy

In [ ]:
!python spider_nest.py -s > 'spider_nest_logs' 2>&1

In [ ]:
tmp_df = pd.read_csv('tmp_urls.csv', dtype={'id':object, 'urls':object, 'desc_html':object})

Проверяю как запарсилось

In [ ]:
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets

In [ ]:
#HTML(tmp_df.loc[10, 'desc_html'])

Далее джойним html обратно

In [ ]:
#В этой ячейке случалось страшное
#пришлось приводить столбцы к инту потом обратно к стр, что джойн сработал
#После перезапуска ядра все работает и так Оо.

telegram_raw.id = telegram_raw.id.astype(str)
tmp_df.id = tmp_df.id.astype(str)

telegram_raw = telegram_raw.join(tmp_df.loc[:,['id', 'desc_html']].set_index('id'), on='id')

Теперь можно парсить текст, html и теги что бы получить информацию для анализа.

Мои критерии

Достаем работодателя, местоположение, удаленка или нет, опыт работы, телефон.

In [ ]:
employe_r = r'Компания: |'
area_r = r'Город|офисе в|офис в'
phone_r = r'\+\d{11}|8\d{10}'

Листалка вакансий

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
button_2 = widgets.Button(description="Показать вакансии")
input_0 = widgets.IntSlider(min=0,max=len(telegram_raw),step=1)
output_2 = widgets.Output()

display(button_2, input_0, output_2)

counter_2 = 0
def on_trait_change_0(b):
    global counter_2
    counter_2 = b['new']['value']
    
def on_clicked_2(b):
    with output_2:
        try:
            global counter_2
            clear_output()
            print('Строка № ', counter_2)
            display(telegram_raw.iloc[counter_2][['channel','text']])
            counter_2 += 1
            if counter_2 == len(telegram_raw):
                counter_2 = 0
                print('Вакансии закончились')
        except Exception as e:
            print(e)
            
button_2.on_click(on_clicked_2)
input_0.observe(on_trait_change_0)

На этом этапе мне стало понятно, что, из не структурированного текста, получить названия компаний и их местоположения, подходом в лоб через re не так то просто. Потребуется как то разметить текст и много времени с экспериментами. По этому я остановлюсь на том что могу сделать в рамках дз.  А именно - посмотрю подходящие вакансии и немного проанализую каналы.

In [ ]:
def parse_phone(row):
    phone_r = r'\+\d{11}|8\d{10}'
    tmp = re.findall(phone_r, row['text'])
    if len(tmp) > 0:
        return tmp
    return pd.NA

In [ ]:
telegram_raw['phone'] = telegram_raw.apply(parse_phone, axis=1)

### Анализ каналов и вакансий

Смотрим как распределяются виды оставленных контактов по каналам

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
tmp_cols = ['mentions', 'urls', 'emails', 'phone']
contacts_by_channels = telegram_raw.groupby('channel').count().loc[:, tmp_cols]

In [ ]:
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 18}

matplotlib.rc('font', **font)

In [ ]:
contacts_by_channels.plot.bar(figsize=(16,8))

Видно что в основном оставляют телеграм и ссылки, мыло редко и телефон совсем редко. Воможно с помощью телефона оставляют свой телеграм.

#### Поиск подхожящих вакансий

Отбираю вакансии аналитика и смотрю какая часть вакансий на канале подходит мне.

In [ ]:
def search_tags(row, tag):
    tag = tag.lower().replace(' ','_')
    for t in row['tags']:
        if not (re.search(tag, t.lower()) is None):
            return True
    return False

In [ ]:
def search_text(row, tag):
    tag = tag.lower()
    return not (re.search(tag, row['text'].lower()) is None)

In [ ]:
def search_desc_html(row, tag):
    tag = tag.lower()
    return not (re.search(tag, str(row['desc_html']).lower()) is None)

In [ ]:
def search_all(row, tag):
    tmp = False
    if(not (pd.isnull(row['text']))):
        tmp = search_text(row, tag)
    if(not pd.isna(row['text'])):
        tmp = tmp|search_text(row, tag)
    if(not pd.isna(row['desc_html'])):
        tmp = tmp|search_desc_html(row, tag)
    return tmp

Интересующие меня направления и скилы

In [ ]:
import itertools
#all_analytic = r'analyst|analytic|аналитик|Data Analyst|BI Aналитик|DA Aналитик|bi|business intelligence|биай'
#da_analytic = r'Data Analyst|DA Aналитик|Дата Аналитик|Data '
#bi_analytic = r'BI Aналитик|bi|business intelligence|биай'
a = ['analyst','analytic','аналитик','analitic']
d = ['data', 'da', 'дата', 'дэйта']
b = ['bi', 'business intelligence', 'биэй', 'биай', 'бизнес']

#Беру все перестановки из 2х множеств
da = '|'.join(map(' '.join, itertools.chain(itertools.product(a, d), itertools.product(d, a))))
ba = '|'.join(map(' '.join, itertools.chain(itertools.product(a, b), itertools.product(b, a))))
a_as_min = '|'.join(a)

location = r'remote|удаленка|удаленно|ремоут'
skill0 = r'python'
skill1 = r'sql'
skill2 = r'tableu'
skill3 = r'статистический анализ'

Отбор подходящих вакансий.

In [ ]:
telegram_raw['da'] = telegram_raw.apply(search_all,args=(ba,), axis=1)
telegram_raw['ba'] = telegram_raw.apply(search_all,args=(da,), axis=1)
telegram_raw['any_a'] = telegram_raw.apply(search_all,args=(a_as_min,), axis=1)

In [ ]:
tmp_cols = ['da','ba','any_a']
target_vac_by_channels = telegram_raw.groupby('channel').sum()[tmp_cols]

In [ ]:
target_vac_by_channels.plot.bar(figsize=(20,8))

Теперь видна некоторая статистика каналов относительно видой вакансий аналитик на них. Тк. это всего лишь одна выбрка, она конечно же не репрезентативна. Но далее в таком ключе можно собрать более детальную статистику по каналам, ведя историю вакансий в сообщениях. Так же это поможет выбрать более подходящие для поиска нужной вакансии.

## Вывод

В ходе дз были изучены и закреплены навыки работы с инструментами скрепинга и парсинга, инструментами для работы c json и html. Так же навыки работы с интересными штуками вроде интерфейсов в ipython и др. Изучены основы работы с открытыми api telegram. Способы взаимодействия со внешними скриптами из ноутбука. После сбора данных, были проанализированны каналы с вакансиями в telegram и подобраны вакансии отвечающие критериям поиска. Наболее ценно, что в ходе дз сформировался подходящий для пет проекта материал. К сожалению времени что бы выполнить задание в полной мере не хватило. По этому работа может выглядеть сжатой и скачущей, в общем недоделанной. За что извиняюсь и надеюсь что выполнил ее в достаточной мере.

PS:У меня на машине ноутбук запускается беЯз ошибок, надеюсь после переноса на другую все будет работать. Вроде все ссылки были условные.